In [1]:
import os
import sys
import torch
import unittest
from torchvision import transforms
from graph_bridges.data.dataloaders_utils import load_dataloader
from graph_bridges.models.reference_process.reference_process_utils import load_reference

from graph_bridges.configs.spin_glass.spin_glass_config_sb import SBConfig
from graph_bridges.data.spin_glass_dataloaders_config import ParametrizedSpinGlassHamiltonianConfig
from graph_bridges.models.reference_process.reference_process_config import GlauberDynamicsConfig

from graph_bridges.models.backward_rates.ctdd_backward_rate_config import BackRateMLPConfig
from graph_bridges.models.reference_process.ctdd_reference import GaussianTargetRate
from graph_bridges.models.reference_process.glauber_reference import GlauberDynamics
from graph_bridges.models.schedulers.scheduling_sb import SBScheduler
from graph_bridges.configs.spin_glass.spin_glass_config_sb import SBConfig

from graph_bridges.data.transforms import SpinsToBinaryTensor
spins_to_binary = SpinsToBinaryTensor()

In [2]:
sb_config = SBConfig(experiment_indentifier="test_glauber")
sb_config.data = ParametrizedSpinGlassHamiltonianConfig(batch_size=2,
                                                       number_of_spins=3,
                                                       data="small_spins")
sb_config.reference = GlauberDynamicsConfig()
sb_config.initialize_new_experiment()

device = torch.device("cpu")

# obtain dataloaders
data_dataloader = load_dataloader(sb_config, type="data", device=device)
target_dataloader = load_dataloader(sb_config, type="target", device=device)

# obtain data
x_adj_data = next(data_dataloader.train().__iter__())[0].to(device)
x_adj_target = next(target_dataloader.train().__iter__())[0].to(device)
batch_size = x_adj_data.shape[0]
times = torch.rand(batch_size,device=device)

reference_process = GlauberDynamics(sb_config, device)
scheduler = SBScheduler(sb_config, device)
tau = reference_process.tau

Coupling from Spin File Different from Config
Coupling from Spin File Different from Config


In [62]:
S = sb_config.data.S
batch_size,number_of_spins = states_spins.shape
transition_rates_ = torch.zeros(batch_size,number_of_spins,S)

states_spins = x_adj_data

all_flip_rates = reference_process.all_flip_rates(states_spins)

batch_index = torch.arange(0,batch_size)
spin_site_index = torch.arange(0,number_of_spins)

states_index = spins_to_binary(x_adj_data).long() # where each site is 
states_flip_index = (~states_index.bool()).long() # where each site is going 

transition_rates_[repeated_batch_index,repeated_spin_site_index,flatten_states_index] = 1./tau - flatten_all_flip_rates
transition_rates_[repeated_batch_index,repeated_spin_site_index,flatten_states_flip_index] = flatten_all_flip_rates

In [63]:
states_index

tensor([[1, 1, 1],
        [1, 0, 0]])

In [37]:
batch_index

tensor([0, 1])

In [56]:
repeated_batch_index = torch.repeat_interleave(batch_index,number_of_spins)
repeated_spin_site_index = spin_site_index.repeat(batch_size)
flatten_states_index = states_index.flatten()
flatten_states_flip_index = states_flip_index.flatten()
flatten_all_flip_rates = all_flip_rates.flatten()

In [53]:
transition_rates_[1,]

tensor([[0.0000, 0.5909],
        [0.5087, 0.0000],
        [0.5064, 0.0000]], grad_fn=<SelectBackward0>)